# Test Tokenizer
   
This notebook is to show examples of how to use the `tokenizer`

## Add Lexos to the Jupyter `sys.path`

In [1]:
%run jupyter_local_setup.py ../../../lexos

System path set to `../../../lexos`.


## Import Lexos Modules

In [2]:
from lexos.cutter import Ginsu
from lexos.cutter import Machete
from lexos import tokenizer
from lexos.io.smart import Loader

## Load data

In [21]:
data = "../test_data/txt/Austen_Pride.txt"
loader = Loader()
loader.load(data)
dataDoc = tokenizer.make_doc(loader.texts[0],model="en_core_web_sm")

## Cutting with Ginsu
Ginsu is used for cutting docs

### Ginsu split_doc()
Splits doc by segmenting it into sections of n tokens

In [61]:
cutter = Ginsu()
pride_segments = cutter.split_doc(dataDoc,n=7500) #returns a list of docs
#arguments:
#doc = doc to be split
#n = number of tokens per segment (default = 1000)
#merge_threshold (float) = percentage of n tokens below which the last segment will be merged (default = 0.5)
#overlap (int) = number of tokens to overlap (default = 0)


### Ginsu split()
Functions the same as split_doc() except it takes a list of docs and returns a list of lists of docs

### Ginsu splitn()
split a list of docs by desired number of segments

In [19]:
pride_segments = cutter.splitn([dataDoc],n=10) #returns a list of lists of docs
#n = number of segments to make
#has the same additional arguments as split())

10

## Cutting with Machete
Machete is used for cutting texts

### Machete split()

In [60]:
cutter = Machete()
dataText = loader.texts[0]
segments = cutter.split(dataText,n= 7500) #returns a list of str lists
#Accepts a text or list of texts to cut
#Contains the same extra arguments as Ginsu.split() with two additional arguments
#tokenizer (str) = The name of the tokenizer function to use (default = none)
#as_string (bool) = whether to return the segments as a list of strings (default = True)

### Machete splitn()
split a list of texts by desired number of segments

In [57]:
segments = cutter.splitn(dataText,n= 10) #returns a list of str lists
print(len(segments[0]))

10
